In [ ]:
%%javascript
import notebook
notebook.nbextensions.check_nbextension('nb-mermaid',user=True)
require(['base/js/utils'],
function(utils) {
        utils.load_extensions('nb-mermaid/nb-mermaid');
});

# TM1 in the DevOps Ecosystem with TM1Py
The below code snippets cover on a step-by-step basis on how to implement tm1-git methodology in your existing tm1 implementation.<br>
What's needed in order to run this:

- Python >= 3.7
- TM1py >= 1.10.0
- Git installed
- Git repository (GitHub, Azure, AWS Code Commit)
- Two tm1 instances up and running

### Brief review of Git related concepts:
Before jumping into the demo, let's try to level the ground on some basic concepts. What is...?

- ``Git``: <br>
Git is a free and open-source distributed version control system designed to track changes in code and other files, allowing multiple developers to work on a project simultaneously. It provides a structured and efficient way to manage and collaborate on software development by keeping track of code history, enabling branching and merging, and facilitating collaboration among team members.

- ``Repository``: <br>
A repository, often referred to as a "repo," is a storage location where all the files and version history of a project are stored. It can be thought of as a folder or directory that contains all the code, documents, and assets for a specific project.

- ``Initialization``: <br>
Initialization typically refers to the process of setting up a new repository or project with version control using Git. When you initialize a repository, Git creates a hidden directory called ".git" in the project's root directory. This directory contains all the necessary files and data to track changes and manage version history for your project.

- ``Branch``: <br>
A branch in Git is a separate line of development that diverges from the main codebase, allowing multiple developers to work on different features, fixes, or experiments simultaneously. Each branch represents a copy of the project's code, and changes made in one branch do not immediately affect other branches. Branches are used to isolate and manage code changes, making it easier to collaborate on and organize the development process.

- ``Commit``: <br>
A commit is a snapshot of the changes made to the files in a Git repository at a specific point in time. Each commit represents a set of changes (additions, deletions, or modifications) made by a developer. Commits are accompanied by a commit message that describes the purpose or context of the changes.

- ``Push``:<br>
Pushing in Git refers to the action of uploading your local commits to a remote repository. It is typically used to share your local changes with others or to synchronize your work with a central repository. When you push, your commits become part of the remote repository's history.

- ``Pull``:<br>
Pulling in Git is the process of fetching and merging changes from a remote repository into your local repository. It's used to update your local copy of a repository with changes made by other collaborators. A "pull" combines the "fetch" (retrieving changes) and "merge" (combining changes) operations into a single step.

- ``Pull Request``:<br>
A pull request (often abbreviated as PR) is a feature primarily associated with Git hosting platforms like GitHub and GitLab. It is a way for developers to propose changes to a repository's codebase. When you create a pull request, you are requesting that the repository's owner or maintainers review and potentially merge your changes into the main branch. Pull requests provide a collaborative and structured way to manage code contributions.

- ``GitHub``: <br>
GitHub is a web-based platform and hosting service that enhances collaboration and version control for software development projects. It leverages the Git version control system to provide a centralized location for developers to store, manage, and collaborate on code repositories. GitHub offers a wide range of features to facilitate code sharing, issue tracking, code review, and project management.

### Brief review of Jenkins related concepts:
Before jumping into the demo, let's try to level the ground on some basic concepts. What is...?

- ``Jenkins``: <br>
Jenkins is an open-source automation server that is widely used in DevOps and software development processes. It provides a platform for automating various tasks related to building, testing, and deploying software. Jenkins helps streamline the development and delivery pipeline by enabling continuous integration and continuous delivery (CI/CD) practices.

- ``Pipeline``: <br>
Jenkins Pipelines are a key feature that enable you to define and manage your CI/CD processes as code. They offer a structured way to automate and orchestrate the steps involved in building, testing, and deploying software. Jenkins Pipelines are typically written in a domain-specific language called Groovy and can be version-controlled alongside your application code.

## What's our starting point?
- An empty GitHub repository
- TM1 Instance in PROD that we want to source control
- A Jenkins agent install in the TM1 Server

### Import necessary modules, define global variables and Git functions
Below we will define the source instance, other global variables and Git related functions we will use to manage our assets through Git

In [ ]:
# importing the required libraries
import configparser
import json
from TM1py.Services import TM1Service
from TM1py.Objects import GitProject

# setting up OS varibles
config = configparser.ConfigParser()
config.read('config.ini')

# defining tm1 source and target instance from config.ini
tm1_instance_prod = 'DEMO_PROD'

# define git variables
git_url='https://github.com/nicolasbisurgi/horizon2023_demo.git'
git_user = 'nicolasbisurgi'
git_email ='nicolasbisurgi@gmail.com'

# Define Git functions

def initiate_git_repo(tm1_instance_name:str, deployment: str):
    with TM1Service(**config[tm1_instance_name]) as tm1:

        # get instance configuration fields
        git_pat = config.get(tm1_instance_name, 'git_pat')

        # initiate git repo
        tm1.git.git_init(
            git_url=git_url,
            deployment=deployment,
            username=git_user,
            password=git_pat,
            force=True
        )

def push_to_git_repo(tm1_instance_name:str, message: str, branch: str, new_branch:str= None):
    with TM1Service(**config[tm1_instance_name]) as tm1:

        # get instance configuration fields
        git_pat = config.get(tm1_instance_name, 'git_pat')

        # push to repo
        tm1.git.git_push(
            message=message,
            author=git_user,
            email=git_email,
            branch=branch,
            new_branch=new_branch,
            force=True,
            username=git_user,
            password=git_pat,
            execute=True
        )

def pull_from_git_repo(tm1_instance_name: str, branch: str):

    with TM1Service(**config[tm1_instance_name]) as tm1:
        # get instance configuration fields
        git_pat = config.get(tm1_instance_name, 'git_pat')

        # pull to repo
        pull_plan = tm1.git.git_pull(
            branch=branch,
            force=True,
            execute=True,
            username=git_user,
            password=git_pat
        )
    # the below is not needed to execute the function but it offers some visibility
    # to what's happening between the git repo and the tm1 server
    pull_plan_ID = json.loads(pull_plan.content).get('ID')
    pull_plan_operations = json.loads(pull_plan.content).get('Operations')

    print(
        f"Created plan ID: {pull_plan_ID} with the below list of objects:")
    for operation in pull_plan_operations:
        print(f"{operation}")


def put_tm1project(tm1_instance_name:str):
    with TM1Service(**config[tm1_instance_name]) as tm1:

        # put tm1project on target instance
        tm1_project = GitProject.TM1Project.from_file('tm1project.json')
        tm1.git.tm1project_put(tm1_project)

### Initiate Git repository in TM1
Action GitInit binds a remote Git repository to the TM1 server. It also initializes a Git context that stores the information of the Git-related operations, for example, the most recent Git branch that has been deployed. The Git context is persistent and server-scoped.


In [ ]:
# Initiate instances
initiate_git_repo(tm1_instance_prod, deployment='PROD')

``` mermaid
flowchart LR
subgraph "TM1 Server"
    A[(TM1_Instance)] -->|new directory| D["`./}git/.git`"]
end
subgraph "GitHub"
    A[(TM1_Instance)] <-->|REST| G(((Git Repo)))
end
```

### Customizing your tm1 assets with TM1Project
In order to custom select what we want to migrate and how we want to do so, we can make use of the TM1 project<br>

A project file tm1project.json SHOULD be created for each model. It specifies how to deploy and publish the model. The content of the project file is a JSON object with properties explained in the following sections.<br>

The project file can be manually modified on Git. The server exposes the project file as a !tm1project resource, which can be viewed (GET) and modified (PUT). The project file is a part of the Git context. When a model is successfully deployed, the project file in the deployed model becomes the project file of the server.<br>

*Below is a sample of the tm1project that we will be using in our session*

```json
{
  "Version": "1.0",
  "Name": "horizon2023_demo",
  "Ignore":
  [
    "Cubes",
    "Dimensions",
    "!Processes('}bedrock*')"
  ],
  "Files":
  [
    "source_files/rules/*.*",
    "source_files/data/*.*",
    "source_files/metadata/*.*",
    "source_files/misc/*.*",
    "source_files/scripts/*.*"
  ],
  "Tasks":
  {
    "pre_push":
    {
      "Process": "Processes('sys_pre_push')"
    },
    "post_push":
    {
      "Process": "Processes('sys_post_push')"
    },
    "pre_pull":
    {
      "Process": "Processes('sys_pre_pull')"
    },
    "post_pull":
    {
      "Process": "Processes('sys_post_pull')"
    }
  },
  "PrePush":
  [
    "Tasks('pre_push')"
  ],
  "PostPush":
  [
    "Tasks('post_push')"
  ],
  "PrePull":
  [
    "Tasks('pre_pull')"
  ],
  "PostPull":
  [
    "Tasks('post_pull')"
  ]
}
```

##### Tasks as hooks
The below TI's will handle code packaging activities as well as pre and post deployment steps as follows:
- ``sys_pre_push``: This process will export rules, attributes, hierarchies and (some) data to flat files before pushing to the Git repo.
- ``sys_post_push``: This process will be execute immediately after pushing to the repo, there are no actions in it for now.
- ``sys_pre_pull``: This process will run savedataall, take a backup of the instance and clean the logs.
- ``sys_post_pull``: This process will load the files generated by the source with the `sys_pre_push` process into the target tm1 instance.

All these processes are wrapper os bedrock code, but you can put whatever you want in them. 

In [ ]:
# put the tm1project
put_tm1project(tm1_instance_prod)

### Push source code to remote Git repo
The first step to publish a model is to execute the GitPush action to create a Git push plan.

GitPush takes the following parameters:

- ``Branch``: the branch from which the last commit is used as the parent commit of the commit to create. If the Git repository is empty, this parameter **MUST NOT** be specified.

- ``NewBranch``: if specified, the server creates a new branch and push the new commit onto it; if not specified, the server pushes the new commit onto the branch specified by Branch. If the Git repository is empty, this parameter MUST be present to instruct the server to create a base commit.

- ``Force``: a flag passed in for evaluating preconditions.

- ``Message``: the commit message.

- ``Author``: the name of the commit author.

- ``Email``: the email of the commit author.

- ``Username``: the Git credential.

- ``Password``: the Git credential.

In [ ]:
push_to_git_repo(
    tm1_instance_name=tm1_instance_prod,
    message='initial_commit',
    branch='main'
)

``` mermaid
flowchart TD
subgraph "GitHub"
    G(((Git Repo)))
end
subgraph "TM1 Prod Server"
    B[(TM1_Instance)] <-->|"`REST<br> pushes code to repo as defined in tm1project`"| G
    B[(TM1_Instance)] --> |changes since init or last push| D["`./}git/.git`"]
end
```

### Bug to Fix & Enhancement
Create new instance from Jenkins

### Create a Development and Support instances
In order to do bug fixing and new development we need 2 new instances

``` mermaid
flowchart LR
subgraph "GitHub"
    G(((Git Repo)))
end
subgraph "TM1 Server"
    J(((Jenkins Agent))) --> | creates service | T[(TM1_Instance)]
    T <--> | pulls from repo | G
end  
subgraph "Jenkins Server"
    S[(Jenkins Server)] --> | executes Pipeline | J
end
```

#### Bug Fix first then push to GitHub

In [ ]:
tm1_instance_support = 'DEMO_SUPPORT'
push_to_git_repo(
    tm1_instance_name=tm1_instance_support,
    message='adding comments',
    branch='main',
    new_branch='bugfix_ti_change'
)

#### Graphical representation of current branches in GitHub
``` mermaid
%%{init: { 'logLevel': 'debug', 'theme': 'base', 'themeVariables': {
        'git0': '#ffffff',
        'git1': '#ff0000'},
'gitGraph': {'showBranches': true, 'showCommitLabel':true,'mainBranchName': 'main'}} }%%
gitGraph
       commit id: " " tag: "initial_commit"
       branch bugfix_ti_change
       checkout bugfix_ti_change
       commit id: "added_comment"
```

#### Enhancement and then push to GitHub
Create new process

In [ ]:
tm1_instance_dev = 'DEMO_DEV'
push_to_git_repo(
    tm1_instance_name=tm1_instance_dev,
    message='adding new super empty process',
    branch='main',
    new_branch='feature_new_process'
)

#### Graphical representation of current branches in GitHub
``` mermaid
%%{init: { 'logLevel': 'debug', 'theme': 'base', 'themeVariables': {
        'git0': '#ffffff',
        'git1': '#ff0000',
        'git2': '#00ff00'},
'gitGraph': {'showBranches': true, 'showCommitLabel':true,'mainBranchName': 'main'}} }%%
gitGraph
       commit id: " " tag: "initial_commit"
       branch bugfix_ti_change
       checkout bugfix_ti_change
       commit id: "added_comment"
       checkout main
       branch feature_new_process
       checkout feature_new_process
       commit id: "new_ti"
```

## What's next?

### Merging all changes
Now that both the bug and the enhancement is there, we can merge them either one-by-one to ``main`` or combine them into one and then mergin with ``main``.
LEt's explore this last option.

``` mermaid
%%{init: { 'logLevel': 'debug', 'theme': 'base', 'themeVariables': {
        'git0': '#ffffff',
        'git1': '#ff0000',
        'git2': '#00ff00'},
'gitGraph': {'showBranches': true, 'showCommitLabel':true,'mainBranchName': 'main'}} }%%
gitGraph
       commit id: " " tag: "initial_commit"
       branch bugfix_ti_change
       checkout bugfix_ti_change
       commit id: "added_comment"
       checkout main
       branch feature_new_process
       checkout feature_new_process
       commit id: "added_ti"
       merge bugfix_ti_change tag: "merging bugs and features"
       checkout main
       merge feature_new_process tag: "merging with main"
```

## Overall workflow diagram

``` mermaid
flowchart LR
subgraph "Dev"
    subgraph "TM1 Dev Server"
        U[Developer] --> |makes changes to TI| A[(TM1_Instance)]
        A -->|tracks changes| D["`./}git/.git`"]
    end
end
subgraph "GitHub"
    D -->|"`REST<br>pushes as single commit`"| G(((Git Repo)))
    G --> |run security checks on code first | G
end
subgraph "Ops"
    subgraph "Jenkins Server"
        G -->|webhook| S[(Jenkins Server)]
    end
    subgraph "TM1 Prod Server"
        S --> | executes Pipeline | J(((Jenkins Agent)))
        J --> |get ready to deploy|B[(TM1_Instance)]
        B -.  pulls from repo .-> G
    end

end
```

### Missing automations
This is a short demo so not all features of these apps are shown; but one of the key pieces to mention is that we could:
- Set GitHub to protect a specific branch so any merge happens through a code review process
- Set GitHub to scan our code for hardcoded passords
- Set GitHub to call Jenkins through a 'webhook' everytime our 'main' branch is updated
- Jenkins would not just automatically deploy to production, but it would also updte the support instance for immediate remediation (if needed)
- Jenkins can run server and environmental checks before and after deployement
- Jenkins pipeline could also be part of a Muticonfiguration pipeline that would be called when source system have been deployed
- Launch RushTi scripts to test the recently deployed code
- Call DataSync to update cubes


## Why take this approach?
Taking this approach to development will not be natural for TM1 developre in your organization, most TM1 developers have not worked with Git. So why making this cultural shift to a DevOps approach?
Aside from all the obvious benefits mentioned before... we are sort of forced to:
1. Enterprise direction: most big organizations have this implemented already and it's a matter of time since they demand TM1 behaves as all the other enterprise tool.
2. IBM direction: this approach, though not 100% identical, is they approach that IBM is taking with v12. There will no longer be files to move, nor scripts to run inside the server (as there are no more servers). This is future prove. 

### Extended use for DevOps applications
You can wrap-up the above functions in a command-line app that can be called by other DevOps applications such as Jenkins pipelines.

``` bash
Usage:
  git.py init (--instance <instance> --user <user> --password <password> --deployment <deployment>)
  git.py pull (--instance <instance> --user <user> --password <password> --branch <branch>)
  git.py push (--instance <instance> --user <user> --password <password> --branch <branch> --message <message> --email <email>) [--new_branch <new_branch>]
  git.py -h | --help
  git.py -v | --version

Examples:
  git.py init (--instance "tm1_instance_support" --user "nbisurgi" --password 12345 --deployment "support")
  git.py pull --instance "tm1_instance_prod" --user "nbisurgi" --password 12345 --branch "main"
  git.py push --instance "tm1_instance_dev" --user "nbisurgi" --password 12345 --branch "main" --new_branch "dev"

Options:
  --instance <instance>         # Target instance for process to run.
  --deployment <deployment>     # Deployment (DEV, QA, ..., PROD).
  --user <user>                 # GitHub username
  --password <password>         # GitHub Personal Access Token.
  --branch <branch>             # Branch to work with (pull from or push to)
  --message <message>           # Commit message
  --email <email>               # Email to validate in GitHub
  --new_branch <branch>         # New branch (leave blank if no new branch needs to be created)
  -h --help                     # Show this screen.
  -v --version                  # Show version.

```